# Imports & Constants

In [23]:
import copy
import os

import pandas as pd
import urllib

pd.set_option('display.max_colwidth', -1)


In [2]:
TARGET_CELL_TYPE = 'Hepg2'
DATA_DIR = "../../dat/inp"

# Load data and sanity check it

In [3]:
cols_df = pd.read_csv(os.path.join(DATA_DIR, 'deepsea_cols.tsv'), sep='\t')
urls = pd.read_csv(os.path.join(DATA_DIR, "deepsea_peak_urls.csv"), header=0, names=["url"]).values.squeeze()
target_tfs_df = pd.read_csv(os.path.join(DATA_DIR, "encode_hepg2_deepsea_cols.csv"))

In [4]:
target_tfs_df.drop_duplicates(subset=('Data Source', 'Cell Type', 'TF/DNase/Histone', 'Treatment'))

,Data Source,Cell Type,TF/DNase/Histone,Treatment,AUC
0,ENCODE,HepG2,CTCF,None,0.984069
4,ENCODE,HepG2,EZH2,None,0.952970
5,ENCODE,HepG2,ATF3,None,0.983841
6,ENCODE,HepG2,BHLHE40,None,0.968259
8,ENCODE,HepG2,CEBPB,None,0.975537
9,ENCODE,HepG2,CEBPB,forskolin,0.972264
11,ENCODE,HepG2,CEBPD,None,0.970669
12,ENCODE,HepG2,ELF1,None,0.964535
13,ENCODE,HepG2,FOSL2,None,0.954806
14,ENCODE,HepG2,FOXA1,None,0.943877


In [5]:
cols_df = cols_df.drop_duplicates(subset=('Data Source', 'Cell Type', 'TF/DNase/Histone', 'Treatment'))

In [6]:
def url_matches(url, row):
    url = url.lower()
    return (
        row['Cell Type'].lower() in url and
        row['TF/DNase/Histone'].lower() in url
    )

def find_urls(row):
    matching_urls = []
    for url in urls:
        if url_matches(url, row): matching_urls.append(url)
    return matching_urls


cols_with_urls = list()
i = 0
for idx, row in cols_df.iterrows():
    matching_urls = find_urls(row)
    for url in matching_urls:
        new_row = copy.copy(row)
        new_row['url'] = url
        cols_with_urls.append(new_row)
        
cols_urls_df = pd.DataFrame(cols_with_urls)

In [7]:
cols_urls_df[(cols_urls_df['Cell Type'] == 'HepG2') & (cols_urls_df['TF/DNase/Histone'] == 'CTCF')]

,Data Source,Cell Type,TF/DNase/Histone,Treatment,AUC,url
12,ENCODE,HepG2,CTCF,None,0.984068752,http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/wgEncodeAwgTfbsBroadHepg2CtcfUniPk.narrowPeak.gz
12,ENCODE,HepG2,CTCF,None,0.984068752,http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/wgEncodeAwgTfbsHaibHepg2Ctcfsc5916V0416101UniPk.narrowPeak.gz
12,ENCODE,HepG2,CTCF,None,0.984068752,http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/wgEncodeAwgTfbsUtaHepg2CtcfUniPk.narrowPeak.gz
12,ENCODE,HepG2,CTCF,None,0.984068752,http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeAwgTfbsUniform/wgEncodeAwgTfbsUwHepg2CtcfUniPk.narrowPeak.gz


The TFs DF is generated using the `filter_deepsea_cols.py` script. Eventually I should probably just combine these two scripts together, but for now, given the time crunch, I'm going to leave things as-is.

In [14]:
final_tfs_df = target_tfs_df.merge(
    cols_urls_df,
    how='inner',
    on=('Data Source', 'Cell Type', 'TF/DNase/Histone', 'Treatment')
)

In [21]:
final_tfs_df = final_tfs_df.drop_duplicates(subset=('Data Source', 'Cell Type', 'TF/DNase/Histone', 'Treatment'))

In [22]:
len(final_tfs_df)

41

In [27]:
print("TFs to pass to mutagenesis")
tfs_str = ""
for idx, row in final_tfs_df.iterrows():
    cell_type, feature, url = row['Cell Type'], row['TF/DNase/Histone'], row['url']
    fpath = os.path.join(DATA_DIR, f"{cell_type}_{feature}.gz")
#     urllib.request.urlretrieve(url, fpath)
    tfs_str += f"{feature} "
print(tfs_str)

TFs to pass to mutagenesis
CTCF EZH2 ATF3 BHLHE40 CEBPB CEBPB CEBPD ELF1 FOSL2 FOXA1 FOXA2 HDAC2 HNF4A HNF4A HNF4G MBD4 MYBL2 NFIC RXRA SP1 SP2 SRF TAF1 TCF12 TEAD4 USF1 YY1 ZBTB33 ZBTB7A ARID3A BRCA1 CHD2 HSF1 IRF3 MAZ RFX5 SMC3 TBP TCF7L2 USF2 ZNF274 
